In [10]:
import torch, re
from math import floor
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectKBest, f_regression

from AnalyzeTools.preprocess import preprocessData
from AnalyzeTools.superModels import DEEPAR, TFT, RNN

params_path = './Models'

# 가격 데이터

- 소
1. ./beef/경략가격집계 - 소,돼지.csv
2. ./beef/축산유통정보 - 소비자가격.csv
3. ./beef/축평원_소 수입 소매가격.xlsx
4. ./beef/축평원_한우 소매가격(등심∙설도, 1등급 기준).xlsx

- 돼지
1. ./pork/(중)경략가격집계 - 소,돼지.csv
2. /pork/(중)축산유통정보 - 소비자가격.csv
3. /pork/축평원_돼지 삼겹살 소매가격.xlsx
4. /pork/축평원_돼지수입 삼겹살 소매가격.xlsx

# (중)경략가격집계 - 소,돼지.csv --> Predict 돼지 온도체

In [2]:
# ''' Read data '''
# raw_file_name = '경략가격집계 - 소,돼지'
# product = "pork-hot"
# product_attribute = "경락가격"

# _output = 'MAX_COST_AMT' # MIN_COST_AMT
# default_exclude_cols = ['JUDGE_GUBN', 'JUDGE_BREED', 'JUDGE_SEX', 'SABLE_GUBN', 'ABATT_CODE']

# df = pd.read_csv('../Data/pork/경략가격집계 - 소,돼지.csv', encoding = 'euc_kr', engine ='python').query("JUDGE_KIND == 2")

# df = df.drop(default_exclude_cols, axis=1)
# df = df.groupby(['STD_DATE']).mean().reset_index()
# df['STD_DATE'] = df['STD_DATE'].apply(lambda x: "20" + "-".join(x.split("/")))

# df, _input = preprocessData(df, 'STD_DATE', _output)

# 경략가격집계 - 소,돼지.csv --> Predict 돼지 냉도체

In [3]:
# ''' Read data '''
# params_path = './Models'
# raw_file_name = '경략가격집계 - 소,돼지'
# product = "pork-ice"
# product_attribute = "경락가격"

# _output = 'MAX_COST_AMT' # MIN_COST_AMT
# default_exclude_cols = ['JUDGE_GUBN', 'JUDGE_BREED', 'JUDGE_SEX', 'SABLE_GUBN', 'ABATT_CODE']

# df = pd.read_csv('../Data/pork/경략가격집계 - 소,돼지.csv', encoding = 'euc_kr', engine ='python').query("JUDGE_KIND == 5")

# df = df.drop(default_exclude_cols, axis=1)
# df = df.groupby(['STD_DATE']).mean().reset_index()
# df['STD_DATE'] = df['STD_DATE'].apply(lambda x: "20" + "-".join(x.split("/")))

# df, _input = preprocessData(df, 'STD_DATE', _output)

# (중)축산유통정보 - 소비자가격.csv --> Predict 돼지갈비(CTSED_CODE == 4304)

In [4]:
# ''' Read data '''
# raw_file_name = '(중)축산유통정보 - 소비자가격'
# product = "pork-hot"
# product_attribute = "경락가격"

# _output = 'DLPC' # MIN_COST_AMT
# default_exclude_cols = [
#     'SN', 'INPUT_MTHD_CODE', 'ENTRP_CODE', 'PRDLST_CODE', 'GRAD_CODE', 'GOODS_TPCD', 'GOODS_NM',
#     'UNIT', 'HIST_NO', 'PRICE_ERROR_YN', 'LNM', 'USE_YN', 'REGISTER', 'REGISTER_ID', 'UPDUSR', 
#     'UPDUSR_ID', 'UPDDE', 'TRN_ID', 'MSG_ID', 'TRN_STATS', 'ERROR_REASON', 'KEYWORD_ERROR_YN', 
#     'PCFLT_TIME', 'RGSDE', 'TRN_MSG', 'INSPECT_YN', 'INSPECT_DATE', 'INSPECT_YN2', 'INSPECT_DATE2', 
#     'BMS_NO', 'MGR_ABATT_CODE', 'ABATT_CODE', 'POSTN_SPRT_CODE'
# ]

# df = pd.read_csv('../Data/pork/(중)축산유통정보 - 소비자가격.csv', encoding = 'euc_kr', engine ='python')

# df = df.drop(default_exclude_cols, axis=1).query("CTSED_CODE == 4304") # specific 품종

# df = df.groupby(['TRN_DT']).mean().reset_index()
# df['TRN_DT'] = df['TRN_DT'].apply(lambda x: "20" + "-".join(x.split("/")))

# df, _input = preprocessData(df, 'TRN_DT', _output)


# 축평원_돼지 삼겹살 소매가격.xlsx -> Predict 돼지_삼겹살 소매가격

In [5]:
# ''' Read data '''
# raw_file_name = '축평원_돼지 삼겹살 소매가격' 
# product = "삼겹살"
# product_attribute = "소매가격"

# _output = '평균'
# default_exclude_cols = []

# df = pd.read_excel(f'../Data/pork/{raw_file_name}.xlsx', '돼지_삼겹살 소매가격').iloc[1:, :5]

# df.columns = ['year', 'month-day', '평균', '최고', '최저']

# df['year'] = df['year'].interpolate(method='pad')
# df['date'] = df['year'] + " " + df['month-day']
# df['date'] = df['date'].apply(lambda x: '-'.join(re.findall("\d+", x)))

# df = df[['date', '평균', '최고', '최저']]

# df = df.groupby(['date']).mean().reset_index()
# df, _input = preprocessData(df, 'date', _output)

There are too few features in the data. The raw data features will be used.


# 축평원_돼지수입 삼겹살 소매가격.xlsx -> Predict 수입돼지고기 소비자가격

In [12]:
# ''' Read data '''
# raw_file_name = '축평원_돼지수입 삼겹살 소매가격' 
# product = "수입_돼지고기"
# product_attribute = "소매가격"

# _output = '평균'
# default_exclude_cols = []

# df = pd.read_excel(f'../Data/pork/{raw_file_name}.xlsx', 'Sheet1').iloc[1:, :5]
# df.columns = ['year', 'month-day', '평균', '최고', '최저']

# df['year'] = df['year'].interpolate(method='pad')
# df['date'] = df['year'] + " " + df['month-day']
# df['date'] = df['date'].apply(lambda x: '-'.join(re.findall("\d+", x)))

# df = df[['date', '평균', '최고', '최저']]

# df = df.groupby(['date']).mean().reset_index()

# df, _input = preprocessData(df, 'date', _output)

There are too few features in the data. The raw data features will be used.


# 경략가격집계 - 소,돼지.csv --> Predict 소

In [ ]:
# ''' Read data '''
# params_path = './Models'
# raw_file_name = '경략가격집계 - 소,돼지'
# product = "beef"
# product_attribute = "경락가격"

# _output = 'MAX_COST_AMT' # MIN_COST_AMT
# default_exclude_cols = ['JUDGE_GUBN', 'JUDGE_BREED', 'JUDGE_SEX', 'SABLE_GUBN', 'ABATT_CODE']

# df = pd.read_csv('../Data/beef/경략가격집계 - 소,돼지.csv', encoding = 'euc_kr', engine ='python').query("JUDGE_KIND == 1")

# df = df.drop(default_exclude_cols, axis=1)
# df = df.groupby(['STD_DATE']).mean().reset_index()
# df['STD_DATE'] = df['STD_DATE'].apply(lambda x: "20" + "-".join(x.split("/")))

# df, _input = preprocessData(df, 'STD_DATE', _output)

# 농협축산정보센터_월별 한우 산지가격.xlsx --> Predict 한우

In [ ]:
# ''' Read data '''
# params_path = './Models'
# raw_file_name = '농협축산정보센터_월별 한우 산지가격' 
# product = "Korea_beef"
# product_attribute = "산지가격"

# _output = '산지가격' # MIN_COST_AMT
# default_exclude_cols = []

# df1 = pd.read_excel(f'../Data/beef/{raw_file_name}.xlsx', '1999-2009')
# df1 = df1[df1.columns[:9]][3:]
# df1 = df1.replace(0, np.nan)
# df1.rename(columns={'구분': 'date'}, inplace=True)

# df2 = pd.read_excel(f'../Data/beef/{raw_file_name}.xlsx', '2010-2022')
# df2 = df2[df2.columns[:9]][3:]
# df2 = df2.replace(0, np.nan)
# df2.rename(columns={'구분': 'date'}, inplace=True)

# df1[df1.columns[1:]] = df1[df1.columns[1:]].interpolate(method='linear', limit_direction='both')
# df2[df2.columns[1:]] = df2[df2.columns[1:]].interpolate(method='linear', limit_direction='both')

# df1['산지가격'] = df1[df1.columns[1:]].mean(axis=1)
# df2['산지가격'] = df2[df2.columns[1:]].mean(axis=1)

# df1 = df1[['date', _output]]
# df2 = df2[['date', _output]]

# df = pd.concat((df1, df2), axis=0)
# del df1, df2

# df['date'] = pd.to_datetime(df['date'])
# df = df.sort_values('date')
# df.index = range(len(df))

# df = df.drop(default_exclude_cols, axis=1)
# df = df.groupby(['date']).mean().reset_index()

# df, _input = preprocessData(df, 'date', _output)

# group = ['group']
# time_varying_known_categoricals = ['month', 'week']
# time_varying_unknown_categoricals = []
# time_varying_known_reals = ['time_idx']
# time_varying_unknown_reals = _input + [_output]

# 축산유통정보 - 소비자가격.csv --> Predict 소

In [ ]:
# # 축산유통정보 - 소비자가격.csv

# ''' Read data '''
# params_path = './Models'
# raw_file_name = '축산유통정보 - 소비자가격'
# product = "beef"
# product_attribute = "소비자가격"

# _output = 'DLPC' 

# df = pd.read_csv('../Data/beef/축산유통정보 - 소비자가격.csv', encoding = 'euc_kr', engine ='python')

# default_exclude_cols = [
#     'SN', 'INPUT_MTHD_CODE', 'ENTRP_CODE', 'PRDLST_CODE', 'GRAD_CODE', 'GOODS_TPCD', 'GOODS_NM',
#     'UNIT', 'HIST_NO', 'PRICE_ERROR_YN', 'LNM', 'USE_YN', 'REGISTER', 'REGISTER_ID', 'UPDUSR', 
#     'UPDUSR_ID', 'UPDDE', 'TRN_ID', 'MSG_ID', 'TRN_STATS', 'ERROR_REASON', 'KEYWORD_ERROR_YN', 
#     'PCFLT_TIME', 'RGSDE', 'TRN_MSG', 'INSPECT_YN', 'INSPECT_DATE', 'INSPECT_YN2', 'INSPECT_DATE2', 
#     'BMS_NO', 'MGR_ABATT_CODE', 'ABATT_CODE', 'POSTN_SPRT_CODE'
# ]
# df = df.drop(default_exclude_cols, axis=1).query("CTSED_CODE == 4301") # specific 품종

# df = df.groupby(['TRN_DT']).mean().reset_index()
# df['TRN_DT'] = df['TRN_DT'].apply(lambda x: "20" + "-".join(x.split("/")))

# df, _input = preprocessData(df, 'TRN_DT', _output)

# 축평원_소 수입 소매가격.xlsx --> Predict 미국산_갈비

In [ ]:
# ''' Read data '''
# raw_file_name = '축평원_소 수입 소매가격' 
# product = "미국산_갈비"
# product_attribute = "수입 소매가격"

# _output = '평균'
# default_exclude_cols = []

# df = pd.read_excel(f'../Data/beef/{raw_file_name}.xlsx', '미국산_갈비').iloc[1:, :5]
# df.columns = ['year', 'month-day', '평균', '최고', '최저']

# df['year'] = df['year'].interpolate(method='pad')
# df['date'] = df['year'] + " " + df['month-day']
# df['date'] = df['date'].apply(lambda x: '-'.join(re.findall("\d+", x)))

# df = df[['date', '평균', '최고', '최저']]

# df = df.groupby(['date']).mean().reset_index()

# df, _input = preprocessData(df, 'date', _output)

# group = ['group']
# time_varying_known_categoricals = ['month', 'week']
# time_varying_unknown_categoricals = []
# time_varying_known_reals = ['time_idx']
# time_varying_unknown_reals = _input + [_output]

# 축평원_한우 소매가격(등심∙설도, 1등급 기준).xlsx --> 한우 등심 1+등급 소매가격

In [ ]:
# ''' Read data '''
# raw_file_name = '축평원_한우 소매가격(등심∙설도, 1등급 기준)' 
# product = "한우 등심_1+등급"
# product_attribute = "소매가격"

# _output = '평균'
# default_exclude_cols = []

# df = pd.read_excel(f'../Data/beef/{raw_file_name}.xlsx', '등심_1+등급').iloc[1:, :5]

# df.columns = ['year', 'month-day', '평균', '최고', '최저']

# df['year'] = df['year'].interpolate(method='pad')
# df['date'] = df['year'] + " " + df['month-day']
# df['date'] = df['date'].apply(lambda x: '-'.join(re.findall("\d+", x)))

# df = df[['date', '평균', '최고', '최저']]

# df = df.groupby(['date']).mean().reset_index()

# df, _input = preprocessData(df, 'date', _output)

# group = ['group']
# time_varying_known_categoricals = ['month', 'week']
# time_varying_unknown_categoricals = []
# time_varying_known_reals = ['time_idx']
# time_varying_unknown_reals = _input + [_output]